<a href="https://colab.research.google.com/github/HimashiRathnayake/CMCS-Text-Classification/blob/main/Adapter_Based_Fine_Tuning/Word_Level_Tasks_Adapter_Based_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
model_type = "SinBERT" #@param ["SinBERT", "Bert", "XLM-R"]
load_adapter = False #@param {type:"boolean"}
train = True #@param {type:"boolean"}
evaluate = True #@param {type:"boolean"}
save_adapter = False #@param {type:"boolean"}
unfreeze_model = False #@param {type:"boolean"}
lang_adapter_setting = "none" #@param ["none", "stack", "parallel"]
adapter_config = "pfeiffer" #@param ["houlsby", "pfeiffer"]
learning_rate = 2e-5 # 2e-5, 5e-4



In [ ]:
pretrained_adapter_path = "/content/drive/Shareddrives/FYP/TrainedAdapters/Final/bert_sinBert_task_adapter_language_identification" #TrainedAdapters/bert_sinBert_task_adapter_language_identification"

In [ ]:
import torch
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets
!pip install seqeval
# !pip install --upgrade torch torchvision

     |████████████████████████████████| 3.8 MB 27.2 MB/s 
     |████████████████████████████████| 895 kB 62.3 MB/s 
     |████████████████████████████████| 67 kB 7.2 MB/s 
     |████████████████████████████████| 6.5 MB 58.4 MB/s 
     |████████████████████████████████| 596 kB 68.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 32.0 MB/s 
     |████████████████████████████████| 1.1 MB 67.0 MB/s 
     |████████████████████████████████| 212 kB 70.3 MB/s 
     |████████████████████████████████| 136 kB 72.5 MB/s 
     |████████████████████████████████| 127 kB 75.5 MB/s 
     |████████████████████████████████| 144 kB 76.3 MB/s 
     |████████████████████████████████| 271 kB 69.3 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [ ]:
tags_ind = ['Sinhala', 'English', 'Sin-Eng', 'Eng-Sin', 'Mixed', 'NameEntity', 'Symbol']

Summarize Dataset

In [ ]:
# import pandas as pd

# url = '/content/drive/MyDrive/FYP/Final/dataset/XLMR/withSemId/xlmr_Final_2.json'

# tags_ind = ['Sinhala', 'English', 'Sin-Eng', 'Eng-Sin', 'Mixed', 'NameEntity', 'Symbol']

# df = pd.read_json(url, lines=True)

# count = 0
# for labels in df['tags']:
#   temp =[]
#   for label in labels:
#     temp.append(tags_ind.index(label))
#   df['tags'][count] = temp
#   count +=1

# df

Split train and test dataset

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

# #split the data into train and test set
# trainData,testData = train_test_split(df, test_size=0.10, random_state=42)

# #save the data
# trainData.to_json('/content/drive/MyDrive/FYP/Final/dataset/Adapter/LID/train_adap_3.json', orient='records', lines=True,  force_ascii=False)
# testData.to_json('/content/drive/MyDrive/FYP/Final/dataset/Adapter/LID/test_adap_3.json', orient='records', lines=True,  force_ascii=False)

Load Dataset

In [ ]:
from datasets import load_dataset, load_metric

dataset = load_dataset('json', data_files={'train': '/content/drive/Shareddrives/FYP/corpus/lang_id_train.json',
                                           'test': '/content/drive/Shareddrives/FYP/corpus/lang_id_test.json'})
dataset

FileNotFoundError: ignored

In [ ]:
dataset['train'][0]

Preprocessing Data

In [ ]:
from transformers import AutoTokenizer

if (model_type=="SinBERT"):
  tokenizer = AutoTokenizer.from_pretrained("/content/drive/Shareddrives/FYP-CodeStars/Implementation/Bert/SinBERT_large", add_prefix_space=True)
elif (model_type=="Bert"):
  tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
else:
  tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

OSError: ignored

In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, padding="max_length", max_length=128)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
tokenized_datasets

Training

In [ ]:
from transformers import AutoConfig, AutoModelWithHeads, AutoModelForTokenClassification

# labels = ['Sinhala', 'English', 'Sin-Eng', 'Eng-Sin', 'Mixed', 'NameEntity', 'Symbol']
# id2label = {id_: label for id_, label in enumerate(labels)}
# label2id = {label: id_ for id_, label in enumerate(labels)}


# if (model_type=="SinBERT"):
#   model = AutoModelWithHeads.from_pretrained("/content/drive/Shareddrives/FYP-CodeStars/Implementation/Bert/SinBERT_large")
# elif (model_type=="Bert"):
#   config = AutoConfig.from_pretrained("bert-base-uncased", num_labels= len(labels))
#   model = AutoModelWithHeads.from_pretrained("bert-base-uncased", config=config)
# else:
#   config = AutoConfig.from_pretrained("xlm-roberta-base", num_labels=len(tags_ind), id2label=id2label, label2id=label2id)
#   model = AutoModelWithHeads.from_pretrained("xlm-roberta-base", config=config)

#model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(tags_ind))

ImportError: ignored

Add Adapters

In [ ]:
from transformers import AdapterConfig

if load_adapter:
  print("loading adapter from", pretrained_adapter_path)
  model.load_adapter(pretrained_adapter_path, with_head=False)

else:
  print("adding new adapter", adapter_config)
  if adapter_config == "pfeiffer":
    config = AdapterConfig.load("pfeiffer", reduction_factor=12)
  else:
    config = AdapterConfig.load("houlsby")
  model.add_adapter("language_identification", config=config)

In [ ]:
#Add a matching classification head
#model.add_classification_head(
#    "language_identification",
#    num_labels=7,
#    id2label={ 0: "Sinhala", 1: "English", 2: "Sin-Eng", 3: "Eng-Sin", 4: "Mixed", 5: "NameEntity", 6: "Symbol"}
#  )

labels = ['Sinhala', 'English', 'Sin-Eng', 'Eng-Sin', 'Mixed', 'NameEntity', 'Symbol']
id2label = {id_: label for id_, label in enumerate(labels)}
model.add_tagging_head("language_identification", num_labels=len(tags_ind), id2label=id2label)

In [ ]:
from transformers.adapters.composition import Fuse, Stack, Parallel

# Without Language Adapters
if lang_adapter_setting == "none":
  model.set_active_adapters("language_identification")

else:
  # Load language adapters
  lang_adapter_config = AdapterConfig.load("pfeiffer+inv")
  model.load_adapter("/content/drive/Shareddrives/FYP/TrainedAdapters/mlm", config=lang_adapter_config, load_as="si-en", with_head=False)
  model.load_adapter("/content/drive/Shareddrives/FYP/TrainedAdapters/si_mlm", config=lang_adapter_config, load_as="si", with_head=False)
  config = AdapterConfig.load("pfeiffer", non_linearity="relu", reduction_factor=2)
  model.load_adapter("en/wiki@ukp", config=config)
  
  # Stack Language Adapters
  if lang_adapter_setting == "stack":
    print("stacking language adapters")
    model.set_active_adapters(Stack("en", "si", "si-en", "language_identification"))

  # Parallel Language Adapters
  else:
    print("stacking parallel language adapters set")
    model.set_active_adapters(Stack(Parallel("en", "si", "si-en"), "language_identification"))

In [ ]:
model.train_adapter("language_identification")

In [ ]:
if unfreeze_model:
  # Unfreeze the model to train both the model and adapter
  model.freeze_model(False)

In [ ]:
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

task = "lid"
args = TrainingArguments(
    #f"test-{task}",
    learning_rate=learning_rate,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    weight_decay=0.01,
    output_dir="/content/drive/MyDrive/FYP/Final/Experiment/Adapters/LID",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    metric_for_best_model="eval_f1",
    load_best_model_at_end=True,
    save_strategy="epoch",
    evaluation_strategy="epoch"
)

In [ ]:
from transformers import DataCollatorForTokenClassification, EarlyStoppingCallback

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [tags_ind[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [tags_ind[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = AdapterTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
if save_adapter:
  technique = "language_identification"
  model.save_adapter("/content/drive/Shareddrives/FYP-CodeStars/Implementation/TrainedAdapters/Final/task_adapter_"+technique, technique)

In [ ]:
# dataset['train'][0]['tokens']

In [ ]:
# from transformers import TokenClassificationPipeline

# classifier = TokenClassificationPipeline(model=model, tokenizer=tokenizer, device=-1)

# classifier(dataset['train'][0]['tokens'])

In [ ]:
# model.save_adapter("./final_adapter", "rotten_tomatoes")

# !ls -lh final_adapter